In [3]:
import numpy as np
import time
from collections import deque
import threading

## Final Request

In [59]:
class Request:
    """Request Object of method: GET POST CONST RAND\n"""
    def __init__(self, method = 'RAND'):
        self.granted = False
        self._method = method
        self.size = None
        
        if self._method == 'GET':
            self.size = np.random.randint(1, 5)
            
        elif self._method == 'POST':
            self.size = np.random.randint(3, 10)
            
        elif self._method == 'CONST':
            self.size = 4
            
        elif self._method == 'RAND':
            self._method = np.random.choice(('GET', 'POST'))
            self.__init__(self._method)
            
        self.src_ip = '.'.join(tuple(map(str,np.random.randint(0, 255, 4))))
        
    def __del__(self):
        #print("Request Complete .. Being deleted.")
        pass

    def __repr__(self):
        return f"Request('{self._method}')"

    def __str__(self):
        return f"Request Object\nMethod: {self._method}\nSize: {self.size}\nGranted: {self.granted}"

## Final Client

In [60]:
class Client:
    """Fires Requests towards the Load Balancer"""
    def __init__(self, deq):
        self._flag = 0
        self._deq = deq
        self._current = None
    
    def _uniform(self, r_type, rps : int): 
        """Uniformly fires Requests of type rtype"""
        sleep_duration = 1 / rps
        while self._flag == 2:
            new_r = Request(r_type)
            #print("U")
            self._deq.appendleft(new_r)
            time.sleep(sleep_duration*0.99)
    
    def _random(self, r_type, rps : int):
        """Fires Requests Randomly of type rtype : On average equals rps"""
        sleep_duration = 1 / rps
        while self._flag == 1:
            fill = min(2*sleep_duration, abs(0.5 * np.random.randn() + sleep_duration))
            #print("R")
            new_r = Request(r_type)
            self._deq.appendleft(new_r)
            time.sleep(fill)
    
    def _single(self, r_type):
        """Fires Single Request of type rtype"""
        new_r = Request(r_type)
        self._deq.appendleft(new_r)
        
    def _fire(self, method, r_type, rps):
        if method == 'stop':
            self._flag = 0
            if self._current:
                self._current.join()
            self._current = None

        elif method == 'uniform':
            self._flag = -1
            if self._current:
                self._current.join()
            self._flag = 2
            self._current = threading.Thread(target = self._uniform, args = (r_type, rps))
            self._current.start()
                
        elif method == "random":
            self._flag = -1
            if self._current:
                self._current.join()
            self._flag = 1
            self._current = threading.Thread(target = self._random, args = (r_type, rps))
            self._current.start()
            
        elif method == 'single':
            self._flag = -1
            if self._current:
                self._current.join()
            self._flag = 3
            self._current = None
            self._single(r_type) # Need thread here ?

    def fire(self, method, rtype = "RAND", rps = 5):
        """method: Type of firing -> 'uniform', 'random', 'single'\n
        rtype: 'RAND', 'GET', 'POST', 'CONST'\n\tdefault RAND\n
        rps: int\n\tdefault 5""" 
        self._fire(method, rtype, rps)
        
    def __str__(self):
        return f"Client Simulator for Deque"

## Final ALBQueue

In [77]:
class ALBQueue(deque):
    def __init__(self):
        super()
    
    def queue_length(self):
        """Current len of queue"""
        return len(self)
    
    def time_exp(self):
        """Time Taken to service Requests proportional to this"""
        return sum([item.size for item in self])
    
    def post_count(self):
        """Count of POST method Requests in Queue"""
        return sum([item._method == 'POST' for item in self])
    
    def get_count(self):
        """Count of GET method Requests in Queue"""
        return sum([item._method == 'GET' for item in self])

In [78]:
x = ALBQueue()

In [79]:
c = Client(x)

In [80]:
c.fire('random', rtype = 'RAND', rps = 3)

In [81]:
c.fire('uniform', rtype = 'RAND', rps = 10)

In [82]:
c.fire('single', rtype = 'POST')

In [83]:
c.fire('stop')

In [84]:
x.queue_length()

10

In [86]:
x.time_exp()

40

In [ ]:
class Client:
    def __init__(self, rps = 5, method = "RAND"):
        self.method = method
        self.rps = rps
        self.flag = None
        
    def _uniform(self):
        sleep_duration = 1 / self.rps
        while self.flag == 'UNFM':
            for _ in range(self.rps):
                new_r = Request(self.method)
                print("U")
                new_deque.append(new_r)
                time.sleep(sleep_duration*0.99)
    
    def _random(self):
        while self.flag == 'RNDM':
            rand_list = np.random.rand(self.rps)
            rand_list /= np.sum(rand_list)
            for sleep_duration in rand_list:
                print("R")
                new_r = Request(self.method)
                new_deque.append(new_r)
                time.sleep(sleep_duration*0.99)
                
    def _flag_setter(self):
        flag = input()
        
        if flag == 'UNFM' and self.flag == 'RNDM':
            self.flag = 'UNFM'
            self.current.join()
            self.current = threading.Thread(target = self._uniform)
            self.current.start()
                               
        elif flag == 'UNFM' and self.flag == 'STOP':
            self.flag = 'UNFM'
            self.current = threading.Thread(target = self._uniform)
            self.current.start()
            
        elif flag == 'RNDM' and self.flag == 'UNFM':
            self.flag = 'RNDM'
            self.current.join()
            self.current = threading.Thread(target = self._random)
            self.current.start()
        
        elif flag == 'RNDM' and self.flag == 'STOP':
            self.flag = 'RNDM'
            self.current = threading.Thread(target = self._random)
            self.current.start()
                               
        elif flag == 'STOP' and (self.flag == 'UNFM' or self.flag == 'RNDM'):
            self.flag = 'STOP'
            self.current.join()
        
    def _checker(self):
        while True:
            self._flag_setter()
        
                               
                               
    def fire(self):
        flag = input()
        if flag == 'UNFM':
            self.flag = 'UNFM'
            self.current = threading.Thread(target = self._uniform)
            self.current.start()
        elif flag == 'RNDM':
            self.flag = 'RNDM'
            self.current = threading.Thread(target = self._random)
            self.current.start()
            
        threading.Thread(target = self._checker).start()

In [ ]:
class Client:
    def __init__(self, rps = 5, method = "RAND"):
        self.method = method
        self.rps = rps
        self.flag = None
        self.looper = True
        
    def _uniform(self):
        sleep_duration = 1 / self.rps
        while self.flag == 'UNFM':
            for _ in range(self.rps):
                new_r = Request(self.method)
                print("U")
                new_deque.append(new_r)
                time.sleep(sleep_duration*0.99)
    
    def _random(self):
        while self.flag == 'RNDM':
            rand_list = np.random.rand(self.rps)
            rand_list /= np.sum(rand_list)
            for sleep_duration in rand_list:
                print("R")
                new_r = Request(self.method)
                new_deque.append(new_r)
                time.sleep(sleep_duration*0.99)
                
    def _flag_setter(self):
        flag = input()
        
        if flag == 'UNFM' and self.flag == 'RNDM':
            self.flag = 'UNFM'
            self.current.join()
            self.current = threading.Thread(target = self._uniform)
            self.current.start()
                               
        elif flag == 'UNFM' and self.flag == 'STOP':
            self.flag = 'UNFM'
            self.current = threading.Thread(target = self._uniform)
            self.current.start()
            
        elif flag == 'RNDM' and self.flag == 'UNFM':
            self.flag = 'RNDM'
            self.current.join()
            self.current = threading.Thread(target = self._random)
            self.current.start()
        
        elif flag == 'RNDM' and self.flag == 'STOP':
            self.flag = 'RNDM'
            self.current = threading.Thread(target = self._random)
            self.current.start()
                               
        elif flag == 'STOP' and (self.flag == 'UNFM' or self.flag == 'RNDM'):
            self.flag = 'STOP'
            self.current.join()
        
        elif flag == 'BRKF':
            if self.flag == 'UNFM' or self.flag == 'RNDM':
                self.flag = None
                self.current.join()
            self.looper = False
        
                               
                               
    def fire(self):
        flag = input()
        while flag not in ('UNFM', 'RNDM'):
            print("Invalid Entry")
            flag = input()
        if flag == 'UNFM':
            self.flag = 'UNFM'
            self.current = threading.Thread(target = self._uniform)
            self.current.start()
        elif flag == 'RNDM':
            self.flag = 'RNDM'
            self.current = threading.Thread(target = self._random)
            self.current.start()
            
        while self.looper:
            self._flag_setter()
            #print(threading.active_count())
        #print("Firing Terminated")